# IBM watsonx.ai + CrewAI Workshop (Refactored)

## Modern Integration with Native CrewAI Support

This notebook demonstrates:

- ✨ **Native CrewAI `LLM` class** - No custom wrappers needed!
- Connect to **IBM watsonx.ai** directly through CrewAI
- Build a **multi-agent workflow** (research, writing, editing crew)
- All powered by watsonx.ai Granite models

## What Changed?

```python
# ❌ Old Way (Custom Wrapper)
class WatsonxCrewAILLM(BaseLLM):
    def __init__(self, model_id, url, ...):
        self._chat = ChatWatsonx(...)
    def call(self, messages, ...):
        # Complex wrapper logic

# ✅ New Way (Native Integration)
from crewai import LLM
llm = LLM(
    model="watsonx/ibm/granite-3-8b-instruct",
    api_key=os.getenv("WATSONX_API_KEY"),
    base_url=os.getenv("WATSONX_URL"),
    project_id=os.getenv("WATSONX_PROJECT_ID")
)
```

## 0. Prerequisites

To run this notebook you need:

- Python 3.10+
- An IBM Cloud account with access to **watsonx.ai**
- A watsonx.ai **service instance**, **project**, and **API key**

> 💡 Run the cells in order from top to bottom the first time.

### Key Benefits of Native Integration

✅ **Fewer dependencies** - No langchain-ibm needed  
✅ **Simpler code** - Direct integration  
✅ **Better maintained** - Official CrewAI support  
✅ **More flexible** - Easy to customize per agent

In [ ]:
# 1) Install required packages - SIMPLIFIED!
# No langchain-ibm, no ibm-watsonx-ai needed!

%pip install -q -U "crewai[tools]" python-dotenv

print("✅ Packages installed!")
print("\n📦 What we installed:")
print("  - CrewAI with native watsonx.ai support")
print("  - Python-dotenv for environment variables")
print("\n✨ No LangChain dependencies needed!")

In [ ]:
# 2) Quick version check

import sys, platform

print("="*70)
print("ENVIRONMENT INFORMATION")
print("="*70)
print(f"Python: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")
print()

try:
    import crewai
    print(f"✅ CrewAI: {crewai.__version__}")
    print("   (with native watsonx.ai support!)")
except Exception as e:
    print(f"❌ CrewAI: {e}")

print("="*70)
print("\n🎉 Ready for native watsonx.ai integration!")

## 1. Configure IBM watsonx.ai Credentials

You'll need three pieces of information:

### 1. IBM Cloud API Key 🔑
- Get from [IBM Cloud IAM](https://cloud.ibm.com/iam/apikeys)
- Used for authentication

### 2. Service URL 🌐
Choose based on your region:

| Region | URL |
|--------|-----|
| Dallas | `https://us-south.ml.cloud.ibm.com` |
| Frankfurt | `https://eu-de.ml.cloud.ibm.com` |
| London | `https://eu-gb.ml.cloud.ibm.com` |
| Tokyo | `https://jp-tok.ml.cloud.ibm.com` |
| Sydney | `https://au-syd.ml.cloud.ibm.com` |

### 3. Project ID 📁
- Found in your watsonx.ai project settings

### Security Best Practices 🛡️

- ✅ Use environment variables
- ✅ Never commit credentials to version control
- ✅ Rotate API keys regularly

In [ ]:
# 3) Configure watsonx.ai credentials

import os
from getpass import getpass

# Disable CrewAI telemetry for cleaner output
os.environ["CREWAI_DISABLE_TELEMETRY"] = "true"
os.environ["CREWAI_TELEMETRY"] = "false"

print("🔐 IBM watsonx.ai Credentials Setup")
print("="*70)

# Get API Key (masked input)
WATSONX_API_KEY = getpass("Enter your IBM Cloud API Key: ")

# Get Service URL
DEFAULT_URL = "https://us-south.ml.cloud.ibm.com"
WATSONX_URL = input(f"Enter watsonx.ai URL [{DEFAULT_URL}]: ").strip() or DEFAULT_URL

# Get Project ID
WATSONX_PROJECT_ID = input("Enter your Project ID: ").strip()

# Store in environment variables
os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY
os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

print("\n✅ Configuration complete!")
print(f"   URL: {WATSONX_URL}")
print(f"   Project ID: {WATSONX_PROJECT_ID[:8]}...")

## 2. Initialize watsonx.ai with CrewAI Native Integration

### The Modern Approach

CrewAI now has **native watsonx.ai support** through its `LLM` class!

#### Benefits:
- ✅ No custom wrapper classes
- ✅ No LangChain dependencies  
- ✅ Cleaner, more maintainable code
- ✅ Official CrewAI support

#### Model Format:
```python
model="watsonx/provider/model-name"

# Examples:
"watsonx/ibm/granite-3-8b-instruct"
"watsonx/ibm/granite-13b-instruct-v2"
"watsonx/meta-llama/llama-3-70b-instruct"
```

In [ ]:
# 4) Initialize watsonx.ai LLM - Native CrewAI Integration!

from crewai import LLM

# Model configuration
WATSONX_MODEL = "watsonx/ibm/granite-3-8b-instruct"

print(f"🤖 Initializing watsonx.ai LLM: {WATSONX_MODEL}")
print("="*70)

# Create the LLM instance - Simple and Clean!
watsonx_llm = LLM(
    model=WATSONX_MODEL,
    api_key=os.environ["WATSONX_API_KEY"],
    base_url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    temperature=0.3,
    max_tokens=512,
)

print("✅ watsonx.ai LLM initialized successfully!")
print("\n📊 Configuration:")
print(f"  Model: {WATSONX_MODEL}")
print(f"  Temperature: 0.3 (balanced)")
print(f"  Max Tokens: 512")
print("  Integration: CrewAI Native")
print("="*70)
print("\n✨ No custom wrappers needed!")

### Quick Connection Test

Let's verify the connection works with a simple test.

In [ ]:
# 5) Test the connection with a simple agent

from crewai import Agent, Task, Crew, Process

print("🧪 Testing watsonx.ai connection...\n")

# Create a simple test agent
test_agent = Agent(
    role="Test Agent",
    goal="Verify watsonx.ai connection",
    backstory="You are a test agent powered by IBM watsonx.ai.",
    llm=watsonx_llm,
    verbose=False,
)

# Simple test task
test_task = Task(
    description="Say hello and confirm you're powered by IBM watsonx.ai. Be brief.",
    expected_output="A short greeting.",
    agent=test_agent,
)

# Run test
test_crew = Crew(
    agents=[test_agent],
    tasks=[test_task],
    process=Process.sequential,
    verbose=False,
)

try:
    result = test_crew.kickoff()
    print("✅ CONNECTION SUCCESSFUL!")
    print("="*70)
    print(result)
    print("="*70)
    print("\n🎉 watsonx.ai is connected and working!")
except Exception as e:
    print(f"❌ Connection failed: {e}")

## 3. Building a Multi-Agent Workflow

We'll build a content creation crew with three specialized agents:

### Agent Pipeline

```
┌──────────────┐
│  Researcher  │ ──▶ Gathers structured notes
└──────┬───────┘
       │
       ▼
┌──────────────┐
│   Writer     │ ──▶ Creates tutorial content
└──────┬───────┘
       │
       ▼
┌──────────────┐
│   Editor     │ ──▶ Polishes final article
└──────────────┘
```

All agents are powered by **IBM watsonx.ai** through CrewAI's native integration!

In [ ]:
# 6) Define the topic for our content creation crew

TOPIC = "Building multi-agent workflows with IBM watsonx.ai and CrewAI"

print("📚 Content Topic:")
print(f"   {TOPIC}")
print("\n💡 Tip: Change the TOPIC variable and re-run to explore different subjects!")

In [ ]:
# 7) Create specialized agents using native watsonx.ai LLM

from crewai import Agent

# Agent 1: Researcher
researcher = Agent(
    role="AI Research Specialist",
    goal="Deeply research the given topic and produce clear, structured notes.",
    backstory=(
        "You are an expert AI research assistant with deep knowledge of IBM watsonx.ai "
        "and enterprise AI systems. You excel at organizing complex information into "
        "concise bullet points that are easy to understand and reuse. Your research "
        "is always thorough, accurate, and well-structured."
    ),
    llm=watsonx_llm,  # Powered by watsonx.ai natively!
    verbose=True,
)

# Agent 2: Technical Writer
writer = Agent(
    role="Technical Writer",
    goal="Turn research notes into an engaging, practical tutorial article.",
    backstory=(
        "You are a patient technical writer who explains advanced AI concepts in "
        "a way that intermediate developers can understand. You have 8 years of "
        "experience creating documentation and tutorials. You know how to structure "
        "content for maximum clarity and include practical examples."
    ),
    llm=watsonx_llm,  # Same LLM, different role!
    verbose=True,
)

# Agent 3: Editor
editor = Agent(
    role="Content Editor",
    goal="Polish content for clarity, accuracy, and workshop readiness.",
    backstory=(
        "You are a meticulous editor who focuses on correctness, structure, and "
        "beginner-friendly language. You have taught hundreds of technical workshops "
        "and know exactly what makes content effective for learning. You're especially "
        "careful about security best practices and practical usability."
    ),
    llm=watsonx_llm,  # All agents use the same watsonx.ai model!
    verbose=True,
)

print("✅ All agents created!")
print("\n🤖 Agent Team:")
print(f"  1. {researcher.role}")
print(f"  2. {writer.role}")
print(f"  3. {editor.role}")
print("\n✨ All powered by IBM watsonx.ai via CrewAI's native integration!")

### Define Tasks with Dependencies

Each task builds on the previous one, creating a sequential workflow.

In [ ]:
# 8) Define tasks with clear expectations

from crewai import Task

# Task 1: Research
research_task = Task(
    description=(
        f"Research the topic: '{TOPIC}'\n\n"
        "Provide:\n"
        "- Overview of watsonx.ai capabilities\n"
        "- Overview of CrewAI framework\n"
        "- 6-8 practical use case ideas\n"
        "- Key benefits and considerations\n"
        "\nFormat: Structured markdown with clear sections."
    ),
    expected_output=(
        "A comprehensive research document (600-800 words) with clear sections "
        "and bullet points."
    ),
    agent=researcher,
)

# Task 2: Writing
writing_task = Task(
    description=(
        "Using the research notes, write a tutorial-style article.\n\n"
        "Include:\n"
        "- Engaging introduction\n"
        "- Architecture explanation\n"
        "- Step-by-step getting started guide\n"
        "- Practical tips and pitfalls\n"
        "- Next steps\n"
        "\nTarget: 800-1200 words, clear and practical."
    ),
    expected_output=(
        "A complete tutorial article with headings, short paragraphs, "
        "and practical examples."
    ),
    agent=writer,
    context=[research_task],  # Uses researcher's output
)

# Task 3: Editing
editing_task = Task(
    description=(
        "Polish the article for a professional workshop audience.\n\n"
        "Focus on:\n"
        "- Improving clarity and flow\n"
        "- Fixing technical inaccuracies\n"
        "- Enhancing structure\n"
        "- Making security practices explicit\n"
        "- Ensuring examples are complete\n"
        "\nFinal output should be workshop-ready."
    ),
    expected_output=(
        "A polished, professional article ready for workshop use. "
        "Error-free and highly educational."
    ),
    agent=editor,
    context=[writing_task],  # Uses writer's output
)

print("✅ Tasks defined with dependencies!")
print("\n📋 Task Pipeline:")
print("  Research → Writing → Editing")

## 4. Execute the Multi-Agent Workflow

Now let's run our complete workflow!

> ⏱️ **Note**: This will take 2-4 minutes as each agent completes its task sequentially.

In [ ]:
# 9) Create and execute the crew

from crewai import Crew, Process

# Assemble the crew
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,
    memory=False,
    verbose=True,
)

print("🚀 Starting content creation workflow...")
print("="*70)
print(f"Topic: {TOPIC}")
print("All agents powered by IBM watsonx.ai Granite models!")
print("This will take a few minutes...")
print("="*70)
print()

# Execute the workflow
result = content_crew.kickoff()

# Extract final content
final_article = str(result)

print("\n\n" + "="*70)
print("🎉 CONTENT CREATION COMPLETE!")
print("="*70)
print("\n📄 Final Article:\n")
print(final_article)
print("\n" + "="*70)
print(f"\n📊 Article Stats:")
print(f"  Length: {len(final_article)} characters")
print(f"  Words: ~{len(final_article.split())}")
print("\n✨ Created using CrewAI's native watsonx.ai integration!")

### Save the Generated Article

In [ ]:
# 10) Save the article to a file

output_file = "watsonx_crewai_article.md"

with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_article)

print(f"✅ Article saved to: {output_file}")
print("📁 You can download this from the file browser.")

## 5. Before & After Comparison

### Old Approach (Custom Wrapper)

```python
# ❌ Complex setup with custom wrapper
from crewai import BaseLLM
from langchain_ibm import ChatWatsonx

class WatsonxCrewAILLM(BaseLLM):
    def __init__(self, model_id, url, project_id, api_key, ...):
        super().__init__(model=model_id, temperature=temperature)
        self._chat = ChatWatsonx(
            model_id=model_id,
            url=url,
            project_id=project_id,
            params={...}
        )
    
    def call(self, messages, tools, callbacks, ...):
        # Complex message handling
        if isinstance(messages, str):
            chat_input = messages
        else:
            # Process message list...
        result = self._chat.invoke(chat_input)
        return getattr(result, "content", str(result))
    
    def supports_function_calling(self):
        return False
    
    def get_context_window_size(self):
        return 8192

# Then use it
llm = WatsonxCrewAILLM(
    model_id="ibm/granite-3-8b-instruct",
    url=WATSONX_URL,
    project_id=WATSONX_PROJECT_ID,
    temperature=0.3,
    max_tokens=512
)
```

**Issues:**
- ~50 lines of wrapper code
- Multiple dependencies (langchain-ibm, ibm-watsonx-ai)
- Complex message handling
- Custom maintenance required

---

### New Approach (Native Integration)

```python
# ✅ Simple, clean, native integration
from crewai import LLM

llm = LLM(
    model="watsonx/ibm/granite-3-8b-instruct",
    api_key=os.environ["WATSONX_API_KEY"],
    base_url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    temperature=0.3,
    max_tokens=512,
)
```

**Benefits:**
- ✅ 7 lines vs 50+ lines
- ✅ No custom wrapper needed
- ✅ Fewer dependencies
- ✅ Official CrewAI support
- ✅ Easier to maintain
- ✅ Better error messages

---

### Code Reduction Summary

| Metric | Old | New | Improvement |
|--------|-----|-----|-------------|
| Lines of wrapper code | ~50 | 0 | -100% |
| Dependencies | 4 | 2 | -50% |
| LLM initialization | ~20 lines | ~7 lines | -65% |
| Complexity | High | Low | Much simpler |
| Maintainability | Custom | Official | Built-in support |

## 6. Experiment: Try Different Topics

The same crew can write about different topics. Just change the `TOPIC` variable!

### Example Topics

1. "Introduction to RAG (Retrieval-Augmented Generation) with watsonx.ai"
2. "Building AI Chatbots with IBM Granite Models"
3. "Enterprise AI Governance and Security Best Practices"
4. "Comparing Different Multi-Agent Frameworks"

To try a new topic:
1. Scroll back to the "Define the topic" cell
2. Change the `TOPIC` variable
3. Re-run from that cell forward

## 7. Summary & Key Takeaways

### What We Learned

1. **Native Integration** - CrewAI's `LLM` class provides seamless watsonx.ai support
2. **No Wrappers Needed** - Direct integration replaces 50+ lines of custom code
3. **Multi-Agent Workflows** - Specialized agents collaborate effectively
4. **Sequential Processing** - Tasks build on each other's outputs
5. **Production Ready** - Official support for enterprise deployments

### Modern Integration Benefits

✅ **Simpler Code** - 7 lines instead of 50+  
✅ **Fewer Dependencies** - No langchain-ibm needed  
✅ **Better Maintained** - Official CrewAI support  
✅ **More Flexible** - Easy to customize  
✅ **Faster Setup** - Get started in minutes  

### Architecture Pattern

```
CrewAI Agent → Native LLM Class → watsonx.ai API
```

No intermediate wrappers or LangChain dependencies!

### Next Steps

- ✨ Experiment with different watsonx.ai models
- 🔧 Add custom tools to agents
- 📊 Build more complex workflows
- 🚀 Deploy to production
- 🎯 Try hierarchical agent structures

### Available watsonx.ai Models

```python
# IBM Granite Models
"watsonx/ibm/granite-3-8b-instruct"      # Efficient, balanced
"watsonx/ibm/granite-13b-instruct-v2"    # Larger, more capable
"watsonx/ibm/granite-3-2b-instruct"      # Smaller, faster

# Meta Llama Models  
"watsonx/meta-llama/llama-3-70b-instruct"  # Very capable
"watsonx/meta-llama/llama-3-8b-instruct"   # Efficient

# Mistral Models
"watsonx/mistralai/mixtral-8x7b-instruct"  # Mixture of experts
```

### Resources

- [CrewAI Documentation](https://docs.crewai.com/)
- [IBM watsonx.ai Documentation](https://www.ibm.com/docs/en/watsonx-as-a-service)
- [IBM Granite Models](https://www.ibm.com/granite)

---

**Thank you for using this refactored notebook!**

🚀 **Happy building with watsonx.ai + CrewAI!** 🚀

---

*This notebook demonstrates CrewAI's native watsonx.ai integration.  
No custom wrappers or LangChain dependencies required!*